HDP-base spell-sound analyzer
developed by Kow Kuroda (kow.kuroda@gmail.com)

History
2024/08/19 Cython を使った cy_gen_ngrams を利用可能にした
2024/09/02 n-gram 生成の generic function を導入し，コードを簡略化

In [1]:
#!pip install -U pyLDAvis
#!pip install -U pandas

In [2]:
## imports
import os, sys
import pprint as pp
import unicodedata

In [3]:
## 一つ上の階層のファイルを見るように設定
sys.path.append(os.path.join(os.path.dirname("__file__"), '..'))

In [4]:
## Cython module の生成 (必要に応じて)
#!python clean setup.py build_ext --inplace

In [5]:
## Cython を使うかどうか
use_Cython = True
if use_Cython:
    %load_ext Cython

In [6]:
## term settings
term_classes        = [ 'spell', 'sound' ]
term_class          = term_classes[0]
ngram_is_inclusive  = True
ngram_inclusiveness = 2
## doc settings
max_doc_size        = 11
min_doc_size        =  3
print(f"max_doc_size: {max_doc_size}")
print(f"min_doc_size: {min_doc_size}")
### boundary handling
add_boundary        = False
boundary_mark       = "#"
## term setting
seg_joint           = ""
gap_mark            = "…"
term_is_skippy      = True
n_for_ngram         = 4
max_gap_ratio       = 0.8
max_gap_size        = round(max_doc_size * max_gap_ratio)
print(f"term_class: {term_class}")
print(f"term_is_skippy: {term_is_skippy}")
print(f"max_gap_size: {max_gap_size}")
print(f"n_for_ngram: {n_for_ngram}")
### accent handling
suppress_accents    = True
accent_marks        = [ "ˈ", "ˌ" ] 
if term_class == 'sound':
    if suppress_accents:
        accent_status = "-unaccented"
    else:
        accent_stratus = "-accented"
else:
    accent_status = ""
print(f"accent_status: {accent_status}")
## define term_type
if term_class == 'spell':
    if term_is_skippy:
        term_type = f"sp_skippy{n_for_ngram}gram"
    else:
        term_type = f"sp_{n_for_ngram}gram"
else:
    if term_is_skippy:
        term_type = f"sn_skippy{n_for_ngram}gram"
    else:
        term_type = f"sn_{n_for_ngram}gram"
## check
print(f"term_type: {term_type}")

max_doc_size: 11
min_doc_size: 3
term_class: spell
term_is_skippy: True
max_gap_size: 9
n_for_ngram: 4
accent_status: 
term_type: sp_skippy4gram


In [7]:
## target language
## a key must be part of a file name 
target_lang_dict = {    'en_US' : 'English (US)',
                        'en_UK' : 'English (UK)',
                        'en_N_only' : 'English noun (WN)',
                        'en_V_only' : 'English verb (WN)',
                        'en_A_only' : 'English adj (WN)',
                        'en_R_only' : 'English adv (WN)',
                        'ar'    : 'Arabic',
                        'de'    : 'German',
                        'de_N_only' : 'German Nouns',
                        'de_non_N_only' : 'German Non-nouns',
                        'eo'    : 'Esperanto',
                        'es_ES' : 'Spanish (Spain)',
                        'es_MX' : 'Spanish (Mexico)',
                        'fi'    : 'Finnish',
                        'fr_FR' : 'French (France)',
                        'fr_QC' : 'French (Quebec)',
                        'is'    : 'Icelandic',
                        'ir'    : 'Irish',
                        'nl'    : 'Dutch',
                        'ro'    : 'Romanian',
                        'sw'    : 'Swahili' }
## proper language selection
target_lang_keys = [    'en_US', # 0
                        'en_UK', # 1
                        'en_N_only', # 2
                        'en_V_only', # 3
                        'en_A_only', # 4
                        'en_R_only', # 5
                        'ar', # 6
                        'de', # 7
                        'de_N_only', # 8
                        'de_non_N_only', # 9
                        'eo', 'es_ES', 'es_MX',
                        'fi', 'fr_FR', 'fr_QC',
                        'is', 'nl', 'ro', 'sw',
                        'ir' # This lacks sound
                    ]
## check
target_lang_key  = target_lang_keys[4]
print(f"target_lang_key: {target_lang_key}")
print(f"target lang: {target_lang_dict[target_lang_key]} [{target_lang_key}]")
## target_attr [effective only for Irish]
target_class = ""
#target_class = None # This causes an unrediable error
if target_lang_key == "ir":
    target_classes = [ 'adjectives', 'nouns', 'verbs' ]
    target_class = f"-{target_classes[3]}"
print(f"target_class: {target_class}")

target_lang_key: en_A_only
target lang: English adj (WN) [en_A_only]
target_class: 


In [8]:
## LDA/HDP
apply_term_filtering = True
## The following parameters need to be relatively large to prevent "Row sum not equal 1" error
term_minfreq       = 2
## The following value is crucial to prevent "Row sum not equal 1" error
abuse_threshold    = 0.04 # larger value selects shorter units, smaller value selects longer units
min_bot_size       = 3
# number of terms listed for a given topic
n_docs_to_show     = 15
n_terms_to_show    = 60
n_terms_to_save    = 120

In [9]:
## sampling
source_sampling          = True
source_sampling_rate     = 0.5
source_sampling_max_size = 5000
second_sampling          = False
second_sampling_rate     = 0.7

In [10]:
## set target files
import glob
data_dir1     = "data/open-dict-ipa/data1"
data_dir2     = "data/open-dict-ipa/data1a"
data_dir3     = "data/wn3"
data_dir4     = "data/irish"
target_files  = glob.glob(f"{data_dir1}/*")
target_files2 = glob.glob(f"{data_dir2}/*")
target_files.extend(target_files2)
target_files3 = glob.glob(f"{data_dir3}/*")
target_files.extend(target_files3)
target_files4 = glob.glob(f"{data_dir4}/*")
target_files.extend(target_files4)
#
target_files = sorted([ file for file in target_files if ".csv" in file ])
pp.pprint(target_files)

['data/irish/word-irish-adjectives-spell.csv',
 'data/irish/word-irish-noun-phrases-spell.csv',
 'data/irish/word-irish-nouns-spell.csv',
 'data/irish/word-irish-possessives-spell.csv',
 'data/irish/word-irish-prepositions-spell.csv',
 'data/irish/word-irish-verbs-spell.csv',
 'data/open-dict-ipa/data1/ar.csv.gz',
 'data/open-dict-ipa/data1/de.csv.gz',
 'data/open-dict-ipa/data1/en_UK.csv.gz',
 'data/open-dict-ipa/data1/en_US.csv.gz',
 'data/open-dict-ipa/data1/eo.csv.gz',
 'data/open-dict-ipa/data1/es_ES.csv.gz',
 'data/open-dict-ipa/data1/es_MX.csv.gz',
 'data/open-dict-ipa/data1/fa.csv.gz',
 'data/open-dict-ipa/data1/fi.csv.gz',
 'data/open-dict-ipa/data1/fr_FR.csv.gz',
 'data/open-dict-ipa/data1/fr_QC.csv.gz',
 'data/open-dict-ipa/data1/is.csv.gz',
 'data/open-dict-ipa/data1/ja.csv.gz',
 'data/open-dict-ipa/data1/jam.csv.gz',
 'data/open-dict-ipa/data1/ma.csv.gz',
 'data/open-dict-ipa/data1/nb.csv.gz',
 'data/open-dict-ipa/data1/nl.csv.gz',
 'data/open-dict-ipa/data1/or.csv.gz',
 '

In [11]:
## get source data from files
import pandas as pd
import gzip
#target_language_key = "en_US" # can be changed to get other languages
#if target_class != "" or target_class is not None:
if target_class != "":
    target_file = [ f for f in target_files if target_lang_key in f and target_class in f ][0]
else:
    target_file = [ f for f in target_files if target_lang_key in f ][0]
print(f"processing: {target_file}")
##
if target_lang_key == "ir":
    col_names = ['spell', 'POS']
else:
    col_names = ['spell', 'sound']
#
if target_file.endswith(".gz"):
    with gzip.open(target_file, "rt") as f:
        raw_df = pd.read_csv(f, encoding = 'utf8', header = None, names = col_names )
else:
    with open(target_file, "rt") as f:
        raw_df = pd.read_csv(f, encoding = 'utf8', header = None, names = col_names )
## normalize characters
raw_df['spell'] = raw_df['spell'].apply(lambda x: unicodedata.normalize('NFC', str(x)))
## modify sound
try:
    sounds = raw_df['sound'].apply(lambda x: x.strip('/') )
    sounds = [ x.split("/,")[0] for x in sounds ] # picks up only the first of multiple entries
    raw_df['sound'] = sounds
except (AttributeError, KeyError):
    pass
#
raw_df.sample(10)

processing: data/wn3/en_A_only.csv


,spell,sound
5115,taxpaying,ˈtækˌspeɪɪŋ
5595,unregistered,ənˈɹɛdʒɪstɝd
0,abandoned,əˈbændənd
980,chirpy,ˈtʃɝpi
1722,doleful,ˈdoʊɫfəɫ
1623,disciplinary,ˈdɪsəpɫəˌnɛɹi
1569,diabetic,ˌdaɪəˈbɛtɪk
1286,coordinating,koʊˈɔɹdəˌneɪtɪŋ
5658,untreated,ənˈtɹitɪd
4944,stillborn,ˈstɪɫˈbɔɹn


In [12]:
## source sampling
len(raw_df)
if source_sampling:
	print(f"source sampling applied")
	if len(raw_df) >= source_sampling_max_size:
		raw_df = raw_df.sample(source_sampling_max_size)
	else:
		raw_df = raw_df.sample(round(len(raw_df) * source_sampling_rate))
## remove accent marking
if suppress_accents:
	try:
		raw_df['sound'] = raw_df['sound'].apply(lambda x: "".join([ y for y in list(x) if y not in accent_marks ]))
	except KeyError:
		pass
## add boudary marks
if add_boundary:
	raw_df['spell'] = raw_df['spell'].apply(lambda x: f"{boundary_mark}{x}{boundary_mark}")
	try:
		raw_df['sound'] = raw_df['sound'].apply(lambda x: f"{boundary_mark}{x}{boundary_mark}")
	except KeyError:
		pass
#
print(raw_df)

source sampling applied
              spell           sound
2164          fifty           fɪfti
3922     perceptual     pɝsɛptʃəwəɫ
2392        gracile          ɡɹæsəɫ
818        bumbling        bəmbəɫɪŋ
1159  computational  kɑmpjuteɪʃənəɫ
...             ...             ...
827       burlesque          bɝɫɛsk
4687        shallow           ʃæɫoʊ
4260      qualified       kwɑɫəfaɪd
795       brotherly         bɹəðɝɫi
3436     moderating       mɑdɝeɪtɪŋ

[5000 rows x 2 columns]


In [13]:
## generate 1-grams for spell and sound
## spell
raw_df['sp_1gram'] = raw_df['spell'].apply(lambda x: list(str(x)))
# add column of size
raw_df['sp_size'] = raw_df['sp_1gram'].apply(lambda x: len(x))
# add column of count of '-' inside
raw_df['hyphen'] = raw_df['sp_1gram'].apply(lambda x: list(x).count("-"))
# add column of count of '.' inside
raw_df['period'] = raw_df['sp_1gram'].apply(lambda x: list(x).count("."))
## sound
# takes the first entry, removes '/' around
try:
    raw_df['sn_1gram'] = raw_df['sound'].apply(lambda x: list(x) )
except (TypeError, KeyError):
    pass
# add column of size
try:
    raw_df['sn_size'] = raw_df['sn_1gram'].apply(lambda x: len(x))
except KeyError:
    pass
## check
raw_df

,spell,sound,sp_1gram,sp_size,hyphen,period,sn_1gram,sn_size
2164,fifty,fɪfti,"[f, i, f, t, y]",5,0,0,"[f, ɪ, f, t, i]",5
3922,perceptual,pɝsɛptʃəwəɫ,"[p, e, r, c, e, p, t, u, a, l]",10,0,0,"[p, ɝ, s, ɛ, p, t, ʃ, ə, w, ə, ɫ]",11
2392,gracile,ɡɹæsəɫ,"[g, r, a, c, i, l, e]",7,0,0,"[ɡ, ɹ, æ, s, ə, ɫ]",6
818,bumbling,bəmbəɫɪŋ,"[b, u, m, b, l, i, n, g]",8,0,0,"[b, ə, m, b, ə, ɫ, ɪ, ŋ]",8
1159,computational,kɑmpjuteɪʃənəɫ,"[c, o, m, p, u, t, a, t, i, o, n, a, l]",13,0,0,"[k, ɑ, m, p, j, u, t, e, ɪ, ʃ, ə, n, ə, ɫ]",14
...,...,...,...,...,...,...,...,...
827,burlesque,bɝɫɛsk,"[b, u, r, l, e, s, q, u, e]",9,0,0,"[b, ɝ, ɫ, ɛ, s, k]",6
4687,shallow,ʃæɫoʊ,"[s, h, a, l, l, o, w]",7,0,0,"[ʃ, æ, ɫ, o, ʊ]",5
4260,qualified,kwɑɫəfaɪd,"[q, u, a, l, i, f, i, e, d]",9,0,0,"[k, w, ɑ, ɫ, ə, f, a, ɪ, d]",9
795,brotherly,bɹəðɝɫi,"[b, r, o, t, h, e, r, l, y]",9,0,0,"[b, ɹ, ə, ð, ɝ, ɫ, i]",7


In [14]:
## filtering raw_data by size
print(f"term_type: {term_type}")
if "sp_" in term_type:
    df_filtered = raw_df[ (raw_df['sp_size'] <= max_doc_size) & (raw_df['sp_size'] >= min_doc_size) & (raw_df['hyphen'] == 0) & (raw_df['period'] == 0) ]
else:
    df_filtered = raw_df[ (raw_df['sn_size'] <= max_doc_size) & (raw_df['sn_size'] >= min_doc_size) ]
#
df_filtered

term_type: sp_skippy4gram


,spell,sound,sp_1gram,sp_size,hyphen,period,sn_1gram,sn_size
2164,fifty,fɪfti,"[f, i, f, t, y]",5,0,0,"[f, ɪ, f, t, i]",5
3922,perceptual,pɝsɛptʃəwəɫ,"[p, e, r, c, e, p, t, u, a, l]",10,0,0,"[p, ɝ, s, ɛ, p, t, ʃ, ə, w, ə, ɫ]",11
2392,gracile,ɡɹæsəɫ,"[g, r, a, c, i, l, e]",7,0,0,"[ɡ, ɹ, æ, s, ə, ɫ]",6
818,bumbling,bəmbəɫɪŋ,"[b, u, m, b, l, i, n, g]",8,0,0,"[b, ə, m, b, ə, ɫ, ɪ, ŋ]",8
591,befitting,bɪfɪtɪŋ,"[b, e, f, i, t, t, i, n, g]",9,0,0,"[b, ɪ, f, ɪ, t, ɪ, ŋ]",7
...,...,...,...,...,...,...,...,...
827,burlesque,bɝɫɛsk,"[b, u, r, l, e, s, q, u, e]",9,0,0,"[b, ɝ, ɫ, ɛ, s, k]",6
4687,shallow,ʃæɫoʊ,"[s, h, a, l, l, o, w]",7,0,0,"[ʃ, æ, ɫ, o, ʊ]",5
4260,qualified,kwɑɫəfaɪd,"[q, u, a, l, i, f, i, e, d]",9,0,0,"[k, w, ɑ, ɫ, ə, f, a, ɪ, d]",9
795,brotherly,bɹəðɝɫi,"[b, r, o, t, h, e, r, l, y]",9,0,0,"[b, ɹ, ə, ð, ɝ, ɫ, i]",7


In [15]:
## define df after second sampling if any
len(df_filtered)
if second_sampling:
    df = df_filtered.sample(round(len(df_filtered) * second_sampling_rate))
else:
    df = df_filtered
len(df)

4452

In [16]:
## generic function for n-gram generation
def add_ngram_to_df(dfx, n_for_ngram: int, var_prefix: str = "", ngram_is_skippy: bool = False, ngram_is_inclusive: bool = ngram_is_inclusive, ngram_inclusiveness: int = ngram_inclusiveness, seg_joint: str = "", use_Cython: bool = False, check: bool = False):
    """
    generic function for adding n-gram column to df with a specified n for ngram
    """
    inclusion_size = (n_for_ngram - ngram_inclusiveness)
    print(f"inclusion_size: {inclusion_size}")
    assert inclusion_size >= 0
    source_var = f"{var_prefix}1gram"
    print(f"source_var: {source_var}")
    unigrams = df[source_var]
    if use_Cython:
        import cy_gen_ngrams
        if ngram_is_skippy:
            ngrams = [ [seg_joint.join(x) for x in cy_gen_ngrams.cy_gen_skippy_ngrams(x, n = n_for_ngram, check = False)] for x in unigrams ]
        else:
            ngrams = [ [seg_joint.join(x) for x in cy_gen_ngrams.cy_gen_ngrams(x, n = n_for_ngram, check = False)] for x in unigrams ]
    else:
        import gen_ngrams
        if ngram_is_skippy:
            ngrams = [ gen_ngrams.gen_skippy_ngrams(x, n = n_for_ngram, sep = "", check = False) for x in unigrams ]
        else:
            ngrams = [ gen_ngrams.gen_ngrams(x, n = n_for_ngram, sep = "", check = False) for x in unigrams ]
    ## 包括的 2gramの生成
    if ngram_is_inclusive:
        assert (n_for_ngram - 1) > 0
        if ngram_is_skippy and n_for_ngram > 2:
            supplement_var = f"{var_prefix}skippy{n_for_ngram - 1}gram"
        else:
            supplement_var = f"{var_prefix}{n_for_ngram - 1}gram"
        print(f"supplement_var: {supplement_var}")
        for i, g in enumerate(ngrams):
            included = [ x for x in list(dfx[supplement_var])[i] if len(x) >= inclusion_size ]
            if len(included) > 0:
                g.extend(included)
    ## 変数の追加
    if ngram_is_skippy:
        added_var = f"{var_prefix}skippy{n_for_ngram}gram"
    else:
        added_var = f"{var_prefix}{n_for_ngram}gram"
    print(f"added_var: {added_var}")
    dfx[added_var] = ngrams
    ## check result
    print(dfx[added_var])

# Spell

In [17]:
## spell 2grams
add_ngram_to_df(df, n_for_ngram = 2, var_prefix = "sp_", ngram_is_skippy = False, ngram_is_inclusive = True, ngram_inclusiveness = ngram_inclusiveness, check = True)

inclusion_size: 0
source_var: sp_1gram
supplement_var: sp_1gram
added_var: sp_2gram
2164                      [fi, if, ft, ty, f, i, f, t, y]
3922    [pe, er, rc, ce, ep, pt, tu, ua, al, p, e, r, ...
2392        [gr, ra, ac, ci, il, le, g, r, a, c, i, l, e]
818     [bu, um, mb, bl, li, in, ng, b, u, m, b, l, i,...
591     [be, ef, fi, it, tt, ti, in, ng, b, e, f, i, t...
                              ...                        
827     [bu, ur, rl, le, es, sq, qu, ue, b, u, r, l, e...
4687        [sh, ha, al, ll, lo, ow, s, h, a, l, l, o, w]
4260    [qu, ua, al, li, if, fi, ie, ed, q, u, a, l, i...
795     [br, ro, ot, th, he, er, rl, ly, b, r, o, t, h...
3436    [mo, od, de, er, ra, at, ti, in, ng, m, o, d, ...
Name: sp_2gram, Length: 4452, dtype: object


/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_25165/2063647876.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx[added_var] = ngrams


In [18]:
## spell 3grams
add_ngram_to_df(df, n_for_ngram = 3, var_prefix = "sp_", ngram_is_skippy = False, ngram_is_inclusive = True, ngram_inclusiveness = ngram_inclusiveness, check = True)

inclusion_size: 1
source_var: sp_1gram
supplement_var: sp_2gram
added_var: sp_3gram
2164       [fif, ift, fty, fi, if, ft, ty, f, i, f, t, y]
3922    [per, erc, rce, cep, ept, ptu, tua, ual, pe, e...
2392    [gra, rac, aci, cil, ile, gr, ra, ac, ci, il, ...
818     [bum, umb, mbl, bli, lin, ing, bu, um, mb, bl,...
591     [bef, efi, fit, itt, tti, tin, ing, be, ef, fi...
                              ...                        
827     [bur, url, rle, les, esq, squ, que, bu, ur, rl...
4687    [sha, hal, all, llo, low, sh, ha, al, ll, lo, ...
4260    [qua, ual, ali, lif, ifi, fie, ied, qu, ua, al...
795     [bro, rot, oth, the, her, erl, rly, br, ro, ot...
3436    [mod, ode, der, era, rat, ati, tin, ing, mo, o...
Name: sp_3gram, Length: 4452, dtype: object


/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_25165/2063647876.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx[added_var] = ngrams


In [19]:
## spell 4grams
add_ngram_to_df(df, n_for_ngram = 4, var_prefix = "sp_", ngram_is_skippy = False, ngram_is_inclusive = True, ngram_inclusiveness = ngram_inclusiveness, check = True)

inclusion_size: 2
source_var: sp_1gram
supplement_var: sp_3gram
added_var: sp_4gram
2164          [fift, ifty, fif, ift, fty, fi, if, ft, ty]
3922    [perc, erce, rcep, cept, eptu, ptua, tual, per...
2392    [grac, raci, acil, cile, gra, rac, aci, cil, i...
818     [bumb, umbl, mbli, blin, ling, bum, umb, mbl, ...
591     [befi, efit, fitt, itti, ttin, ting, bef, efi,...
                              ...                        
827     [burl, urle, rles, lesq, esqu, sque, bur, url,...
4687    [shal, hall, allo, llow, sha, hal, all, llo, l...
4260    [qual, uali, alif, lifi, ifie, fied, qua, ual,...
795     [brot, roth, othe, ther, herl, erly, bro, rot,...
3436    [mode, oder, dera, erat, rati, atin, ting, mod...
Name: sp_4gram, Length: 4452, dtype: object


/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_25165/2063647876.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx[added_var] = ngrams


In [20]:
## spell 5grams
add_ngram_to_df(df, n_for_ngram = 5, var_prefix = "sp_", ngram_is_skippy = False, ngram_is_inclusive = True, ngram_inclusiveness = ngram_inclusiveness, check = True)

inclusion_size: 3
source_var: sp_1gram
supplement_var: sp_4gram
added_var: sp_5gram
2164                   [fifty, fift, ifty, fif, ift, fty]
3922    [perce, ercep, rcept, ceptu, eptua, ptual, per...
2392    [graci, racil, acile, grac, raci, acil, cile, ...
818     [bumbl, umbli, mblin, bling, bumb, umbl, mbli,...
591     [befit, efitt, fitti, ittin, tting, befi, efit...
                              ...                        
827     [burle, urles, rlesq, lesqu, esque, burl, urle...
4687    [shall, hallo, allow, shal, hall, allo, llow, ...
4260    [quali, ualif, alifi, lifie, ified, qual, uali...
795     [broth, rothe, other, therl, herly, brot, roth...
3436    [moder, odera, derat, erati, ratin, ating, mod...
Name: sp_5gram, Length: 4452, dtype: object


/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_25165/2063647876.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx[added_var] = ngrams


In [21]:
## spell skippy 2grams
add_ngram_to_df(df, n_for_ngram = 2, var_prefix = "sp_", ngram_is_skippy = True, ngram_is_inclusive = ngram_is_inclusive, ngram_inclusiveness = ngram_inclusiveness, check = True)

inclusion_size: 0
source_var: sp_1gram
supplement_var: sp_1gram
added_var: sp_skippy2gram
2164    [fi, f…f, f…t, f…y, if, i…t, i…y, ft, f…y, ty,...
3922    [pe, p…r, p…c, p…e, p…p, p…t, p…u, p…a, p…l, e...
2392    [gr, g…a, g…c, g…i, g…l, g…e, ra, r…c, r…i, r…...
818     [bu, b…m, b…b, b…l, b…i, b…n, b…g, um, u…b, u…...
591     [be, b…f, b…i, b…t, b…t, b…i, b…n, b…g, ef, e…...
                              ...                        
827     [bu, b…r, b…l, b…e, b…s, b…q, b…u, b…e, ur, u…...
4687    [sh, s…a, s…l, s…l, s…o, s…w, ha, h…l, h…l, h…...
4260    [qu, q…a, q…l, q…i, q…f, q…i, q…e, q…d, ua, u…...
795     [br, b…o, b…t, b…h, b…e, b…r, b…l, b…y, ro, r…...
3436    [mo, m…d, m…e, m…r, m…a, m…t, m…i, m…n, m…g, o...
Name: sp_skippy2gram, Length: 4452, dtype: object


/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_25165/2063647876.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx[added_var] = ngrams


In [22]:
## spell skippy 3grams
add_ngram_to_df(df, n_for_ngram = 3, var_prefix = "sp_", ngram_is_skippy = True, ngram_is_inclusive = ngram_is_inclusive, ngram_inclusiveness = ngram_inclusiveness, check = True)

inclusion_size: 1
source_var: sp_1gram
supplement_var: sp_skippy2gram
added_var: sp_skippy3gram
2164    [fif, fi…t, fi…y, f…ft, f…f…y, f…ty, ift, if…y...
3922    [per, pe…c, pe…e, pe…p, pe…t, pe…u, pe…a, pe…l...
2392    [gra, gr…c, gr…i, gr…l, gr…e, g…ac, g…a…i, g…a...
818     [bum, bu…b, bu…l, bu…i, bu…n, bu…g, b…mb, b…m…...
591     [bef, be…i, be…t, be…t, be…i, be…n, be…g, b…fi...
                              ...                        
827     [bur, bu…l, bu…e, bu…s, bu…q, bu…u, bu…e, b…rl...
4687    [sha, sh…l, sh…l, sh…o, sh…w, s…al, s…a…l, s…a...
4260    [qua, qu…l, qu…i, qu…f, qu…i, qu…e, qu…d, q…al...
795     [bro, br…t, br…h, br…e, br…r, br…l, br…y, b…ot...
3436    [mod, mo…e, mo…r, mo…a, mo…t, mo…i, mo…n, mo…g...
Name: sp_skippy3gram, Length: 4452, dtype: object


/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_25165/2063647876.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx[added_var] = ngrams


In [23]:
## spell skippy 4grams
add_ngram_to_df(df, n_for_ngram = 4, var_prefix = "sp_", ngram_is_skippy = True, ngram_is_inclusive = ngram_is_inclusive, ngram_inclusiveness = ngram_inclusiveness, check = True)

inclusion_size: 2
source_var: sp_1gram
supplement_var: sp_skippy3gram
added_var: sp_skippy4gram
2164    [fift, fif…y, fi…ty, f…fty, ifty, fif, fi…t, f...
3922    [perc, per…e, per…p, per…t, per…u, per…a, per…...
2392    [grac, gra…i, gra…l, gra…e, gr…ci, gr…c…l, gr…...
818     [bumb, bum…l, bum…i, bum…n, bum…g, bu…bl, bu…b...
591     [befi, bef…t, bef…t, bef…i, bef…n, bef…g, be…i...
                              ...                        
827     [burl, bur…e, bur…s, bur…q, bur…u, bur…e, bu…l...
4687    [shal, sha…l, sha…o, sha…w, sh…ll, sh…l…o, sh…...
4260    [qual, qua…i, qua…f, qua…i, qua…e, qua…d, qu…l...
795     [brot, bro…h, bro…e, bro…r, bro…l, bro…y, br…t...
3436    [mode, mod…r, mod…a, mod…t, mod…i, mod…n, mod…...
Name: sp_skippy4gram, Length: 4452, dtype: object


/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_25165/2063647876.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx[added_var] = ngrams


In [24]:
## spell skippy 5grams
add_ngram_to_df(df, n_for_ngram = 5, var_prefix = "sp_", ngram_is_skippy = True, ngram_is_inclusive = ngram_is_inclusive, ngram_inclusiveness = ngram_inclusiveness, check = True)

inclusion_size: 3
source_var: sp_1gram
supplement_var: sp_skippy4gram
added_var: sp_skippy5gram
2164    [fifty, fift, fif…y, fi…ty, f…fty, ifty, fif, ...
3922    [perce, perc…p, perc…t, perc…u, perc…a, perc…l...
2392    [graci, grac…l, grac…e, gra…il, gra…i…e, gra…l...
818     [bumbl, bumb…i, bumb…n, bumb…g, bum…li, bum…l…...
591     [befit, befi…t, befi…i, befi…n, befi…g, bef…tt...
                              ...                        
827     [burle, burl…s, burl…q, burl…u, burl…e, bur…es...
4687    [shall, shal…o, shal…w, sha…lo, sha…l…w, sha…o...
4260    [quali, qual…f, qual…i, qual…e, qual…d, qua…if...
795     [broth, brot…e, brot…r, brot…l, brot…y, bro…he...
3436    [moder, mode…a, mode…t, mode…i, mode…n, mode…g...
Name: sp_skippy5gram, Length: 4452, dtype: object


/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_25165/2063647876.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx[added_var] = ngrams


In [25]:
## check result
target_vars = [ 'spell' ]
target_vars.extend([ f"sp_{i}gram" for i in range(1,6) ])
target_vars.extend([ f"sp_skippy{i}gram" for i in range(2,6) ])
df[target_vars]

,spell,sp_1gram,sp_2gram,sp_3gram,sp_4gram,sp_5gram,sp_skippy2gram,sp_skippy3gram,sp_skippy4gram,sp_skippy5gram
2164,fifty,"[f, i, f, t, y]","[fi, if, ft, ty, f, i, f, t, y]","[fif, ift, fty, fi, if, ft, ty, f, i, f, t, y]","[fift, ifty, fif, ift, fty, fi, if, ft, ty]","[fifty, fift, ifty, fif, ift, fty]","[fi, f…f, f…t, f…y, if, i…t, i…y, ft, f…y, ty,...","[fif, fi…t, fi…y, f…ft, f…f…y, f…ty, ift, if…y...","[fift, fif…y, fi…ty, f…fty, ifty, fif, fi…t, f...","[fifty, fift, fif…y, fi…ty, f…fty, ifty, fif, ..."
3922,perceptual,"[p, e, r, c, e, p, t, u, a, l]","[pe, er, rc, ce, ep, pt, tu, ua, al, p, e, r, ...","[per, erc, rce, cep, ept, ptu, tua, ual, pe, e...","[perc, erce, rcep, cept, eptu, ptua, tual, per...","[perce, ercep, rcept, ceptu, eptua, ptual, per...","[pe, p…r, p…c, p…e, p…p, p…t, p…u, p…a, p…l, e...","[per, pe…c, pe…e, pe…p, pe…t, pe…u, pe…a, pe…l...","[perc, per…e, per…p, per…t, per…u, per…a, per…...","[perce, perc…p, perc…t, perc…u, perc…a, perc…l..."
2392,gracile,"[g, r, a, c, i, l, e]","[gr, ra, ac, ci, il, le, g, r, a, c, i, l, e]","[gra, rac, aci, cil, ile, gr, ra, ac, ci, il, ...","[grac, raci, acil, cile, gra, rac, aci, cil, i...","[graci, racil, acile, grac, raci, acil, cile, ...","[gr, g…a, g…c, g…i, g…l, g…e, ra, r…c, r…i, r…...","[gra, gr…c, gr…i, gr…l, gr…e, g…ac, g…a…i, g…a...","[grac, gra…i, gra…l, gra…e, gr…ci, gr…c…l, gr…...","[graci, grac…l, grac…e, gra…il, gra…i…e, gra…l..."
818,bumbling,"[b, u, m, b, l, i, n, g]","[bu, um, mb, bl, li, in, ng, b, u, m, b, l, i,...","[bum, umb, mbl, bli, lin, ing, bu, um, mb, bl,...","[bumb, umbl, mbli, blin, ling, bum, umb, mbl, ...","[bumbl, umbli, mblin, bling, bumb, umbl, mbli,...","[bu, b…m, b…b, b…l, b…i, b…n, b…g, um, u…b, u…...","[bum, bu…b, bu…l, bu…i, bu…n, bu…g, b…mb, b…m…...","[bumb, bum…l, bum…i, bum…n, bum…g, bu…bl, bu…b...","[bumbl, bumb…i, bumb…n, bumb…g, bum…li, bum…l…..."
591,befitting,"[b, e, f, i, t, t, i, n, g]","[be, ef, fi, it, tt, ti, in, ng, b, e, f, i, t...","[bef, efi, fit, itt, tti, tin, ing, be, ef, fi...","[befi, efit, fitt, itti, ttin, ting, bef, efi,...","[befit, efitt, fitti, ittin, tting, befi, efit...","[be, b…f, b…i, b…t, b…t, b…i, b…n, b…g, ef, e…...","[bef, be…i, be…t, be…t, be…i, be…n, be…g, b…fi...","[befi, bef…t, bef…t, bef…i, bef…n, bef…g, be…i...","[befit, befi…t, befi…i, befi…n, befi…g, bef…tt..."
...,...,...,...,...,...,...,...,...,...,...
827,burlesque,"[b, u, r, l, e, s, q, u, e]","[bu, ur, rl, le, es, sq, qu, ue, b, u, r, l, e...","[bur, url, rle, les, esq, squ, que, bu, ur, rl...","[burl, urle, rles, lesq, esqu, sque, bur, url,...","[burle, urles, rlesq, lesqu, esque, burl, urle...","[bu, b…r, b…l, b…e, b…s, b…q, b…u, b…e, ur, u…...","[bur, bu…l, bu…e, bu…s, bu…q, bu…u, bu…e, b…rl...","[burl, bur…e, bur…s, bur…q, bur…u, bur…e, bu…l...","[burle, burl…s, burl…q, burl…u, burl…e, bur…es..."
4687,shallow,"[s, h, a, l, l, o, w]","[sh, ha, al, ll, lo, ow, s, h, a, l, l, o, w]","[sha, hal, all, llo, low, sh, ha, al, ll, lo, ...","[shal, hall, allo, llow, sha, hal, all, llo, l...","[shall, hallo, allow, shal, hall, allo, llow, ...","[sh, s…a, s…l, s…l, s…o, s…w, ha, h…l, h…l, h…...","[sha, sh…l, sh…l, sh…o, sh…w, s…al, s…a…l, s…a...","[shal, sha…l, sha…o, sha…w, sh…ll, sh…l…o, sh…...","[shall, shal…o, shal…w, sha…lo, sha…l…w, sha…o..."
4260,qualified,"[q, u, a, l, i, f, i, e, d]","[qu, ua, al, li, if, fi, ie, ed, q, u, a, l, i...","[qua, ual, ali, lif, ifi, fie, ied, qu, ua, al...","[qual, uali, alif, lifi, ifie, fied, qua, ual,...","[quali, ualif, alifi, lifie, ified, qual, uali...","[qu, q…a, q…l, q…i, q…f, q…i, q…e, q…d, ua, u…...","[qua, qu…l, qu…i, qu…f, qu…i, qu…e, qu…d, q…al...","[qual, qua…i, qua…f, qua…i, qua…e, qua…d, qu…l...","[quali, qual…f, qual…i, qual…e, qual…d, qua…if..."
795,brotherly,"[b, r, o, t, h, e, r, l, y]","[br, ro, ot, th, he, er, rl, ly, b, r, o, t, h...","[bro, rot, oth, the, her, erl, rly, br, ro, ot...","[brot, roth, othe, ther, herl, erly, bro, rot,...","[broth, rothe, other, therl, herly, brot

# Sound

In [26]:
## sound 2grams
add_ngram_to_df(df, n_for_ngram = 2, var_prefix = "sn_", ngram_is_skippy = False, ngram_is_inclusive = True, ngram_inclusiveness = ngram_inclusiveness, check = True)

inclusion_size: 0
source_var: sn_1gram
supplement_var: sn_1gram
added_var: sn_2gram
2164                      [fɪ, ɪf, ft, ti, f, ɪ, f, t, i]
3922    [pɝ, ɝs, sɛ, ɛp, pt, tʃ, ʃə, əw, wə, əɫ, p, ɝ,...
2392               [ɡɹ, ɹæ, æs, sə, əɫ, ɡ, ɹ, æ, s, ə, ɫ]
818     [bə, əm, mb, bə, əɫ, ɫɪ, ɪŋ, b, ə, m, b, ə, ɫ,...
591         [bɪ, ɪf, fɪ, ɪt, tɪ, ɪŋ, b, ɪ, f, ɪ, t, ɪ, ŋ]
                              ...                        
827                [bɝ, ɝɫ, ɫɛ, ɛs, sk, b, ɝ, ɫ, ɛ, s, k]
4687                      [ʃæ, æɫ, ɫo, oʊ, ʃ, æ, ɫ, o, ʊ]
4260    [kw, wɑ, ɑɫ, ɫə, əf, fa, aɪ, ɪd, k, w, ɑ, ɫ, ə...
795         [bɹ, ɹə, əð, ðɝ, ɝɫ, ɫi, b, ɹ, ə, ð, ɝ, ɫ, i]
3436    [mɑ, ɑd, dɝ, ɝe, eɪ, ɪt, tɪ, ɪŋ, m, ɑ, d, ɝ, e...
Name: sn_2gram, Length: 4452, dtype: object


/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_25165/2063647876.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx[added_var] = ngrams


In [27]:
## sound 3grams
add_ngram_to_df(df, n_for_ngram = 3, var_prefix = "sn_", ngram_is_skippy = False, ngram_is_inclusive = True, ngram_inclusiveness = ngram_inclusiveness, check = True)

inclusion_size: 1
source_var: sn_1gram
supplement_var: sn_2gram
added_var: sn_3gram
2164       [fɪf, ɪft, fti, fɪ, ɪf, ft, ti, f, ɪ, f, t, i]
3922    [pɝs, ɝsɛ, sɛp, ɛpt, ptʃ, tʃə, ʃəw, əwə, wəɫ, ...
2392    [ɡɹæ, ɹæs, æsə, səɫ, ɡɹ, ɹæ, æs, sə, əɫ, ɡ, ɹ,...
818     [bəm, əmb, mbə, bəɫ, əɫɪ, ɫɪŋ, bə, əm, mb, bə,...
591     [bɪf, ɪfɪ, fɪt, ɪtɪ, tɪŋ, bɪ, ɪf, fɪ, ɪt, tɪ, ...
                              ...                        
827     [bɝɫ, ɝɫɛ, ɫɛs, ɛsk, bɝ, ɝɫ, ɫɛ, ɛs, sk, b, ɝ,...
4687       [ʃæɫ, æɫo, ɫoʊ, ʃæ, æɫ, ɫo, oʊ, ʃ, æ, ɫ, o, ʊ]
4260    [kwɑ, wɑɫ, ɑɫə, ɫəf, əfa, faɪ, aɪd, kw, wɑ, ɑɫ...
795     [bɹə, ɹəð, əðɝ, ðɝɫ, ɝɫi, bɹ, ɹə, əð, ðɝ, ɝɫ, ...
3436    [mɑd, ɑdɝ, dɝe, ɝeɪ, eɪt, ɪtɪ, tɪŋ, mɑ, ɑd, dɝ...
Name: sn_3gram, Length: 4452, dtype: object


/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_25165/2063647876.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx[added_var] = ngrams


In [28]:
## sound 4grams
add_ngram_to_df(df, n_for_ngram = 4, var_prefix = "sn_", ngram_is_skippy = False, ngram_is_inclusive = True, ngram_inclusiveness = ngram_inclusiveness, check = True)

inclusion_size: 2
source_var: sn_1gram
supplement_var: sn_3gram
added_var: sn_4gram
2164          [fɪft, ɪfti, fɪf, ɪft, fti, fɪ, ɪf, ft, ti]
3922    [pɝsɛ, ɝsɛp, sɛpt, ɛptʃ, ptʃə, tʃəw, ʃəwə, əwə...
2392    [ɡɹæs, ɹæsə, æsəɫ, ɡɹæ, ɹæs, æsə, səɫ, ɡɹ, ɹæ,...
818     [bəmb, əmbə, mbəɫ, bəɫɪ, əɫɪŋ, bəm, əmb, mbə, ...
591     [bɪfɪ, ɪfɪt, fɪtɪ, ɪtɪŋ, bɪf, ɪfɪ, fɪt, ɪtɪ, t...
                              ...                        
827     [bɝɫɛ, ɝɫɛs, ɫɛsk, bɝɫ, ɝɫɛ, ɫɛs, ɛsk, bɝ, ɝɫ,...
4687          [ʃæɫo, æɫoʊ, ʃæɫ, æɫo, ɫoʊ, ʃæ, æɫ, ɫo, oʊ]
4260    [kwɑɫ, wɑɫə, ɑɫəf, ɫəfa, əfaɪ, faɪd, kwɑ, wɑɫ,...
795     [bɹəð, ɹəðɝ, əðɝɫ, ðɝɫi, bɹə, ɹəð, əðɝ, ðɝɫ, ɝ...
3436    [mɑdɝ, ɑdɝe, dɝeɪ, ɝeɪt, eɪtɪ, ɪtɪŋ, mɑd, ɑdɝ,...
Name: sn_4gram, Length: 4452, dtype: object


/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_25165/2063647876.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx[added_var] = ngrams


In [29]:
## sound 5grams
add_ngram_to_df(df, n_for_ngram = 4, var_prefix = "sn_", ngram_is_skippy = False, ngram_is_inclusive = True, ngram_inclusiveness = ngram_inclusiveness, check = True)

inclusion_size: 2
source_var: sn_1gram
supplement_var: sn_3gram
added_var: sn_4gram
2164          [fɪft, ɪfti, fɪf, ɪft, fti, fɪ, ɪf, ft, ti]
3922    [pɝsɛ, ɝsɛp, sɛpt, ɛptʃ, ptʃə, tʃəw, ʃəwə, əwə...
2392    [ɡɹæs, ɹæsə, æsəɫ, ɡɹæ, ɹæs, æsə, səɫ, ɡɹ, ɹæ,...
818     [bəmb, əmbə, mbəɫ, bəɫɪ, əɫɪŋ, bəm, əmb, mbə, ...
591     [bɪfɪ, ɪfɪt, fɪtɪ, ɪtɪŋ, bɪf, ɪfɪ, fɪt, ɪtɪ, t...
                              ...                        
827     [bɝɫɛ, ɝɫɛs, ɫɛsk, bɝɫ, ɝɫɛ, ɫɛs, ɛsk, bɝ, ɝɫ,...
4687          [ʃæɫo, æɫoʊ, ʃæɫ, æɫo, ɫoʊ, ʃæ, æɫ, ɫo, oʊ]
4260    [kwɑɫ, wɑɫə, ɑɫəf, ɫəfa, əfaɪ, faɪd, kwɑ, wɑɫ,...
795     [bɹəð, ɹəðɝ, əðɝɫ, ðɝɫi, bɹə, ɹəð, əðɝ, ðɝɫ, ɝ...
3436    [mɑdɝ, ɑdɝe, dɝeɪ, ɝeɪt, eɪtɪ, ɪtɪŋ, mɑd, ɑdɝ,...
Name: sn_4gram, Length: 4452, dtype: object


/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_25165/2063647876.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx[added_var] = ngrams


In [30]:
## sound skippy 2grams
add_ngram_to_df(df, n_for_ngram = 2, var_prefix = "sn_", ngram_is_skippy = True, ngram_is_inclusive = True, ngram_inclusiveness = ngram_inclusiveness, check = True)

inclusion_size: 0
source_var: sn_1gram
supplement_var: sn_1gram
added_var: sn_skippy2gram
2164    [fɪ, f…f, f…t, f…i, ɪf, ɪ…t, ɪ…i, ft, f…i, ti,...
3922    [pɝ, p…s, p…ɛ, p…p, p…t, p…ʃ, p…ə, p…w, p…ə, p...
2392    [ɡɹ, ɡ…æ, ɡ…s, ɡ…ə, ɡ…ɫ, ɹæ, ɹ…s, ɹ…ə, ɹ…ɫ, æs...
818     [bə, b…m, b…b, b…ə, b…ɫ, b…ɪ, b…ŋ, əm, ə…b, ə…...
591     [bɪ, b…f, b…ɪ, b…t, b…ɪ, b…ŋ, ɪf, ɪ…ɪ, ɪ…t, ɪ…...
                              ...                        
827     [bɝ, b…ɫ, b…ɛ, b…s, b…k, ɝɫ, ɝ…ɛ, ɝ…s, ɝ…k, ɫɛ...
4687    [ʃæ, ʃ…ɫ, ʃ…o, ʃ…ʊ, æɫ, æ…o, æ…ʊ, ɫo, ɫ…ʊ, oʊ,...
4260    [kw, k…ɑ, k…ɫ, k…ə, k…f, k…a, k…ɪ, k…d, wɑ, w…...
795     [bɹ, b…ə, b…ð, b…ɝ, b…ɫ, b…i, ɹə, ɹ…ð, ɹ…ɝ, ɹ…...
3436    [mɑ, m…d, m…ɝ, m…e, m…ɪ, m…t, m…ɪ, m…ŋ, ɑd, ɑ…...
Name: sn_skippy2gram, Length: 4452, dtype: object


/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_25165/2063647876.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx[added_var] = ngrams


In [31]:
## sound skippy 3grams
add_ngram_to_df(df, n_for_ngram = 3, var_prefix = "sn_", ngram_is_skippy = True, ngram_is_inclusive = True, ngram_inclusiveness = ngram_inclusiveness, check = True)

inclusion_size: 1
source_var: sn_1gram
supplement_var: sn_skippy2gram
added_var: sn_skippy3gram
2164    [fɪf, fɪ…t, fɪ…i, f…ft, f…f…i, f…ti, ɪft, ɪf…i...
3922    [pɝs, pɝ…ɛ, pɝ…p, pɝ…t, pɝ…ʃ, pɝ…ə, pɝ…w, pɝ…ə...
2392    [ɡɹæ, ɡɹ…s, ɡɹ…ə, ɡɹ…ɫ, ɡ…æs, ɡ…æ…ə, ɡ…æ…ɫ, ɡ…...
818     [bəm, bə…b, bə…ə, bə…ɫ, bə…ɪ, bə…ŋ, b…mb, b…m…...
591     [bɪf, bɪ…ɪ, bɪ…t, bɪ…ɪ, bɪ…ŋ, b…fɪ, b…f…t, b…f...
                              ...                        
827     [bɝɫ, bɝ…ɛ, bɝ…s, bɝ…k, b…ɫɛ, b…ɫ…s, b…ɫ…k, b…...
4687    [ʃæɫ, ʃæ…o, ʃæ…ʊ, ʃ…ɫo, ʃ…ɫ…ʊ, ʃ…oʊ, æɫo, æɫ…ʊ...
4260    [kwɑ, kw…ɫ, kw…ə, kw…f, kw…a, kw…ɪ, kw…d, k…ɑɫ...
795     [bɹə, bɹ…ð, bɹ…ɝ, bɹ…ɫ, bɹ…i, b…əð, b…ə…ɝ, b…ə...
3436    [mɑd, mɑ…ɝ, mɑ…e, mɑ…ɪ, mɑ…t, mɑ…ɪ, mɑ…ŋ, m…dɝ...
Name: sn_skippy3gram, Length: 4452, dtype: object


/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_25165/2063647876.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx[added_var] = ngrams


In [32]:
## sound skippy 4grams
add_ngram_to_df(df, n_for_ngram = 4, var_prefix = "sn_", ngram_is_skippy = True, ngram_is_inclusive = True, ngram_inclusiveness = ngram_inclusiveness, check = True)

inclusion_size: 2
source_var: sn_1gram
supplement_var: sn_skippy3gram
added_var: sn_skippy4gram
2164    [fɪft, fɪf…i, fɪ…ti, f…fti, ɪfti, fɪf, fɪ…t, f...
3922    [pɝsɛ, pɝs…p, pɝs…t, pɝs…ʃ, pɝs…ə, pɝs…w, pɝs…...
2392    [ɡɹæs, ɡɹæ…ə, ɡɹæ…ɫ, ɡɹ…sə, ɡɹ…s…ɫ, ɡɹ…əɫ, ɡ…æ...
818     [bəmb, bəm…ə, bəm…ɫ, bəm…ɪ, bəm…ŋ, bə…bə, bə…b...
591     [bɪfɪ, bɪf…t, bɪf…ɪ, bɪf…ŋ, bɪ…ɪt, bɪ…ɪ…ɪ, bɪ…...
                              ...                        
827     [bɝɫɛ, bɝɫ…s, bɝɫ…k, bɝ…ɛs, bɝ…ɛ…k, bɝ…sk, b…ɫ...
4687    [ʃæɫo, ʃæɫ…ʊ, ʃæ…oʊ, ʃ…ɫoʊ, æɫoʊ, ʃæɫ, ʃæ…o, ʃ...
4260    [kwɑɫ, kwɑ…ə, kwɑ…f, kwɑ…a, kwɑ…ɪ, kwɑ…d, kw…ɫ...
795     [bɹəð, bɹə…ɝ, bɹə…ɫ, bɹə…i, bɹ…ðɝ, bɹ…ð…ɫ, bɹ…...
3436    [mɑdɝ, mɑd…e, mɑd…ɪ, mɑd…t, mɑd…ɪ, mɑd…ŋ, mɑ…ɝ...
Name: sn_skippy4gram, Length: 4452, dtype: object


/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_25165/2063647876.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx[added_var] = ngrams


In [33]:
## sound skippy 5grams
add_ngram_to_df(df, n_for_ngram = 5, var_prefix = "sn_", ngram_is_skippy = True, ngram_is_inclusive = True, ngram_inclusiveness = ngram_inclusiveness, check = True)

inclusion_size: 3
source_var: sn_1gram
supplement_var: sn_skippy4gram
added_var: sn_skippy5gram
2164    [fɪfti, fɪft, fɪf…i, fɪ…ti, f…fti, ɪfti, fɪf, ...
3922    [pɝsɛp, pɝsɛ…t, pɝsɛ…ʃ, pɝsɛ…ə, pɝsɛ…w, pɝsɛ…ə...
2392    [ɡɹæsə, ɡɹæs…ɫ, ɡɹæ…əɫ, ɡɹ…səɫ, ɡ…æsəɫ, ɹæsəɫ,...
818     [bəmbə, bəmb…ɫ, bəmb…ɪ, bəmb…ŋ, bəm…əɫ, bəm…ə…...
591     [bɪfɪt, bɪfɪ…ɪ, bɪfɪ…ŋ, bɪf…tɪ, bɪf…t…ŋ, bɪf…ɪ...
                              ...                        
827     [bɝɫɛs, bɝɫɛ…k, bɝɫ…sk, bɝ…ɛsk, b…ɫɛsk, ɝɫɛsk,...
4687    [ʃæɫoʊ, ʃæɫo, ʃæɫ…ʊ, ʃæ…oʊ, ʃ…ɫoʊ, æɫoʊ, ʃæɫ, ...
4260    [kwɑɫə, kwɑɫ…f, kwɑɫ…a, kwɑɫ…ɪ, kwɑɫ…d, kwɑ…əf...
795     [bɹəðɝ, bɹəð…ɫ, bɹəð…i, bɹə…ɝɫ, bɹə…ɝ…i, bɹə…ɫ...
3436    [mɑdɝe, mɑdɝ…ɪ, mɑdɝ…t, mɑdɝ…ɪ, mɑdɝ…ŋ, mɑd…eɪ...
Name: sn_skippy5gram, Length: 4452, dtype: object


/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_25165/2063647876.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx[added_var] = ngrams


In [34]:
## check df
dropped_vars = [ 'sp_size', 'hyphen', 'period', 'sn_size' ]
if term_class == 'spell':
    extra = [ 'sn_1gram', 'sn_2gram', 'sn_3gram', 'sn_4gram',
             'sn_skippy2gram', 'sn_skippy3gram', 'sn_skippy4gram' ]
    dropped_vars.extend(extra)
    target_vars = [ x for x in df.columns if not x in dropped_vars ]
else:
    extra = [ 'sp_1gram', 'sp_2gram', 'sp_3gram', 'sp_4gram',
             'sp_skippy2gram', 'sp_skippy3gram', 'sp_skippy4gram' ]
    dropped_vars.extend(extra)
    target_vars = [ x for x in df.columns if not x in dropped_vars ]    
#
df[target_vars]

,spell,sound,sp_1gram,sp_2gram,sp_3gram,sp_4gram,sp_5gram,sp_skippy2gram,sp_skippy3gram,sp_skippy4gram,sp_skippy5gram,sn_skippy5gram
2164,fifty,fɪfti,"[f, i, f, t, y]","[fi, if, ft, ty, f, i, f, t, y]","[fif, ift, fty, fi, if, ft, ty, f, i, f, t, y]","[fift, ifty, fif, ift, fty, fi, if, ft, ty]","[fifty, fift, ifty, fif, ift, fty]","[fi, f…f, f…t, f…y, if, i…t, i…y, ft, f…y, ty,...","[fif, fi…t, fi…y, f…ft, f…f…y, f…ty, ift, if…y...","[fift, fif…y, fi…ty, f…fty, ifty, fif, fi…t, f...","[fifty, fift, fif…y, fi…ty, f…fty, ifty, fif, ...","[fɪfti, fɪft, fɪf…i, fɪ…ti, f…fti, ɪfti, fɪf, ..."
3922,perceptual,pɝsɛptʃəwəɫ,"[p, e, r, c, e, p, t, u, a, l]","[pe, er, rc, ce, ep, pt, tu, ua, al, p, e, r, ...","[per, erc, rce, cep, ept, ptu, tua, ual, pe, e...","[perc, erce, rcep, cept, eptu, ptua, tual, per...","[perce, ercep, rcept, ceptu, eptua, ptual, per...","[pe, p…r, p…c, p…e, p…p, p…t, p…u, p…a, p…l, e...","[per, pe…c, pe…e, pe…p, pe…t, pe…u, pe…a, pe…l...","[perc, per…e, per…p, per…t, per…u, per…a, per…...","[perce, perc…p, perc…t, perc…u, perc…a, perc…l...","[pɝsɛp, pɝsɛ…t, pɝsɛ…ʃ, pɝsɛ…ə, pɝsɛ…w, pɝsɛ…ə..."
2392,gracile,ɡɹæsəɫ,"[g, r, a, c, i, l, e]","[gr, ra, ac, ci, il, le, g, r, a, c, i, l, e]","[gra, rac, aci, cil, ile, gr, ra, ac, ci, il, ...","[grac, raci, acil, cile, gra, rac, aci, cil, i...","[graci, racil, acile, grac, raci, acil, cile, ...","[gr, g…a, g…c, g…i, g…l, g…e, ra, r…c, r…i, r…...","[gra, gr…c, gr…i, gr…l, gr…e, g…ac, g…a…i, g…a...","[grac, gra…i, gra…l, gra…e, gr…ci, gr…c…l, gr…...","[graci, grac…l, grac…e, gra…il, gra…i…e, gra…l...","[ɡɹæsə, ɡɹæs…ɫ, ɡɹæ…əɫ, ɡɹ…səɫ, ɡ…æsəɫ, ɹæsəɫ,..."
818,bumbling,bəmbəɫɪŋ,"[b, u, m, b, l, i, n, g]","[bu, um, mb, bl, li, in, ng, b, u, m, b, l, i,...","[bum, umb, mbl, bli, lin, ing, bu, um, mb, bl,...","[bumb, umbl, mbli, blin, ling, bum, umb, mbl, ...","[bumbl, umbli, mblin, bling, bumb, umbl, mbli,...","[bu, b…m, b…b, b…l, b…i, b…n, b…g, um, u…b, u…...","[bum, bu…b, bu…l, bu…i, bu…n, bu…g, b…mb, b…m…...","[bumb, bum…l, bum…i, bum…n, bum…g, bu…bl, bu…b...","[bumbl, bumb…i, bumb…n, bumb…g, bum…li, bum…l…...","[bəmbə, bəmb…ɫ, bəmb…ɪ, bəmb…ŋ, bəm…əɫ, bəm…ə…..."
591,befitting,bɪfɪtɪŋ,"[b, e, f, i, t, t, i, n, g]","[be, ef, fi, it, tt, ti, in, ng, b, e, f, i, t...","[bef, efi, fit, itt, tti, tin, ing, be, ef, fi...","[befi, efit, fitt, itti, ttin, ting, bef, efi,...","[befit, efitt, fitti, ittin, tting, befi, efit...","[be, b…f, b…i, b…t, b…t, b…i, b…n, b…g, ef, e…...","[bef, be…i, be…t, be…t, be…i, be…n, be…g, b…fi...","[befi, bef…t, bef…t, bef…i, bef…n, bef…g, be…i...","[befit, befi…t, befi…i, befi…n, befi…g, bef…tt...","[bɪfɪt, bɪfɪ…ɪ, bɪfɪ…ŋ, bɪf…tɪ, bɪf…t…ŋ, bɪf…ɪ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
827,burlesque,bɝɫɛsk,"[b, u, r, l, e, s, q, u, e]","[bu, ur, rl, le, es, sq, qu, ue, b, u, r, l, e...","[bur, url, rle, les, esq, squ, que, bu, ur, rl...","[burl, urle, rles, lesq, esqu, sque, bur, url,...","[burle, urles, rlesq, lesqu, esque, burl, urle...","[bu, b…r, b…l, b…e, b…s, b…q, b…u, b…e, ur, u…...","[bur, bu…l, bu…e, bu…s, bu…q, bu…u, bu…e, b…rl...","[burl, bur…e, bur…s, bur…q, bur…u, bur…e, bu…l...","[burle, burl…s, burl…q, burl…u, burl…e, bur…es...","[bɝɫɛs, bɝɫɛ…k, bɝɫ…sk, bɝ…ɛsk, b…ɫɛsk, ɝɫɛsk,..."
4687,shallow,ʃæɫoʊ,"[s, h, a, l, l, o, w]","[sh, ha, al, ll, lo, ow, s, h, a, l, l, o, w]","[sha, hal, all, llo, low, sh, ha, al, ll, lo, ...","[shal, hall, allo, llow, sha, hal, all, llo, l...","[shall, hallo, allow, shal, hall, allo, llow, ...","[sh, s…a, s…l, s…l, s…o, s…w, ha, h…l, h…l, h…...","[sha, sh…l, sh…l, sh…o, sh…w, s…al, s…a…l, s…a...","[shal, sha…l, sha…o, sha…w, sh…ll, sh…l…o, sh…...","[shall, shal…o, shal…w, sha…lo, sha…l…w, sha…o...","[ʃæɫoʊ, ʃæɫo, ʃæɫ…ʊ, ʃæ…oʊ, ʃ…ɫoʊ, æɫoʊ, ʃæɫ, ..."
4260,qualified,kwɑɫəfaɪd,"[q, u, a, l, i, f, i, e, d]","[qu, ua, al, li, if, fi, ie, ed, q, u, a, l, i...","[qua, ual, ali, lif, ifi, fie, ied, qu, ua, al...","[qual, uali, alif, lifi, ifie, fied, qua, ual,...","[quali, ualif, alifi, lifie, ified, qual, 

# Analysis

In [35]:
## select data type and define doc_dict
import random
if "sp_" in term_type:
    base_type = "spell"
else:
    base_type = "sound"
doc_dict = { i: x for i, x in enumerate(df[base_type]) }
## check
random.sample(doc_dict.items(), 10)

/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_25165/1181347170.py:9: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random.sample(doc_dict.items(), 10)


[(762, 'known'),
 (3022, 'dim'),
 (1083, 'bone'),
 (427, 'boon'),
 (2861, 'brainwashed'),
 (1089, 'saute'),
 (135, 'super'),
 (1791, 'homeopathic'),
 (855, 'political'),
 (3164, 'bubonic')]

In [36]:
## select bots for analysis
enable_term_change = False # if you want to change term_type to save time and energy
if enable_term_change:
	term_type = 'sp_skippy4gram'
	print(f"(changed) term_type: {term_type}")

## bot stands for 'bag-of-terms', a generalization of 'bag-of-words'
bots = [ x for x in df[term_type] if len(x) > min_bot_size ] # Crucially
import random
random.sample(bots, 1)

[['conc',
  'con…i',
  'con…s',
  'con…e',
  'co…ci',
  'co…c…s',
  'co…c…e',
  'co…is',
  'co…i…e',
  'co…se',
  'c…nci',
  'c…nc…s',
  'c…nc…e',
  'c…n…is',
  'c…n…i…e',
  'c…n…se',
  'c…cis',
  'c…ci…e',
  'c…c…se',
  'c…ise',
  'onci',
  'onc…s',
  'onc…e',
  'on…is',
  'on…i…e',
  'on…se',
  'o…cis',
  'o…ci…e',
  'o…c…se',
  'o…ise',
  'ncis',
  'nci…e',
  'nc…se',
  'n…ise',
  'cise',
  'con',
  'co…c',
  'co…i',
  'co…s',
  'co…e',
  'c…nc',
  'c…n…i',
  'c…n…s',
  'c…n…e',
  'c…ci',
  'c…c…s',
  'c…c…e',
  'c…is',
  'c…i…e',
  'c…se',
  'onc',
  'on…i',
  'on…s',
  'on…e',
  'o…ci',
  'o…c…s',
  'o…c…e',
  'o…is',
  'o…i…e',
  'o…se',
  'nci',
  'nc…s',
  'nc…e',
  'n…is',
  'n…i…e',
  'n…se',
  'cis',
  'ci…e',
  'c…se',
  'ise',
  'co',
  'c…n',
  'c…c',
  'c…i',
  'c…s',
  'c…e',
  'on',
  'o…c',
  'o…i',
  'o…s',
  'o…e',
  'nc',
  'n…i',
  'n…s',
  'n…e',
  'ci',
  'c…s',
  'c…e',
  'is',
  'i…e',
  'se']]

In [37]:
## generate dictionary
from gensim.corpora import Dictionary
diction = Dictionary(bots)
print(diction)
#apply_term_filtering = False
if apply_term_filtering:
    print(f"term filtering applied")
    diction.filter_extremes(no_below = term_minfreq, no_above = abuse_threshold)
else:
    print(f"term filtering not applied")
## check
print(diction)
## generate DTM
corpus = [ diction.doc2bow(bot) for bot in bots if len(bot) > min_bot_size ] # Crucially

Dictionary<198272 unique tokens: ['fi', 'fif', 'fift', 'fif…y', 'fi…t']...>
term filtering applied
Dictionary<96630 unique tokens: ['fi', 'fif', 'fift', 'fi…t', 'fi…y']...>


In [38]:
## HDP (n_topics = 15)
import gensim.models
import pyLDAvis.gensim

max_n_topics = 15
hdp15 = gensim.models.HdpModel(corpus, diction, T = max_n_topics, random_state = 1)
vis_data15 = pyLDAvis.gensim.prepare(hdp15, corpus, diction)
pyLDAvis.display(vis_data15)

In [39]:
## save LDAvis output as a html file
save_LDAvis = True
if save_LDAvis:
	lang_dir_name = target_lang_dict[target_lang_key].split()[0]
	vis_output = f"results/LDAvis/{lang_dir_name}/{target_lang_dict[target_lang_key]}{target_class}-HDP-max_ntop{max_n_topics}-{term_type}{accent_status}.html"
	pyLDAvis.save_html(vis_data15, vis_output)

In [40]:
## topic investigation
import numpy as np
import HDP_helper
reload_module = False
if reload_module:
    import importlib
    importlib.reload(HDP_helper)

target_hdp = hdp15
documents_topics = np.zeros([target_hdp.m_T, len(corpus)])
for doc_id, c in enumerate(corpus):
    for topic_id, prob in target_hdp[c]:
        documents_topics[topic_id][doc_id] = prob

target_hdp.optimal_ordering()
for topic_id, probs in enumerate(documents_topics):
    print(f"==============")
    topic_t = target_hdp.print_topic(topic_id, topn = n_terms_to_show)
    print(f"topic_id {topic_id}: {HDP_helper.reformat_topic (topic_t, n_terms_to_show)}")
    print(f"nonzero count: ", len(probs.nonzero()[0]))
    for doc_id in probs.argsort()[::-1][:n_docs_to_show]:
        doc = doc_dict[doc_id]
        print(f"\t{probs[doc_id]:0.4f}: {doc}")

topic_id 0: 0.0 * co…e + 0.0 * c…i…e + 0.0 * o…i…e + 0.0 * c…u + 0.0 * d…c + 0.0 * c…n…e + 0.0 * o…ti + 0.0 * c…g + 0.0 * tic + 0.0 * ec + 0.0 * co…i + 0.0 * u…c + 0.0 * o…u + 0.0 * ct + 0.0 * r…ti + 0.0 * ive + 0.0 * s…o + 0.0 * c…v + 0.0 * o…y + 0.0 * t…v + 0.0 * u…o + 0.0 * t…ic + 0.0 * on…e + 0.0 * nc + 0.0 * t…ve + 0.0 * o…in + 0.0 * c…in + 0.0 * h…r + 0.0 * c…ng + 0.0 * tive + 0.0 * r…ic + 0.0 * c…n…i + 0.0 * tiv + 0.0 * e…y + 0.0 * e…t…e + 0.0 * co…t + 0.0 * a…t…c + 0.0 * o…e…t + 0.0 * t…s + 0.0 * con + 0.0 * ry + 0.0 * e…v + 0.0 * t…y + 0.0 * m…c + 0.0 * h…l + 0.0 * con…e + 0.0 * p…ic + 0.0 * i…ic + 0.0 * c…i…g + 0.0 * p…ti + 0.0 * b…c + 0.0 * ch + 0.0 * n…in + 0.0 * o…te + 0.0 * h…n + 0.0 * r…te + 0.0 * s…u + 0.0 * d…r + 0.0 * c…ic + 0.0 * s…s
nonzero count:  2137
	0.9984: particulate
	0.9984: constricted
	0.9984: uncollected
	0.9984: informative
	0.9984: appreciated
	0.9984: consonantal
	0.9984: unconcealed
	0.9984: uncontested
	0.9984: unconverted
	0.9983: considerate
	0.998

In [41]:
## save topic structures
#hdp.get_topics() # =/= show_topics()
#hdp.print_topics()
hdp_topics = hdp15.show_topics(num_topics = max_n_topics,
                               num_words = n_terms_to_save, formatted = False)
hdp_dict = { tid: values for tid, values in hdp_topics }
## convert to Pandas dataframe
topics_df = pd.DataFrame.from_dict(hdp_dict)
hdp15_topics_out = f"results/terms-by-topics-raw/{lang_dir_name}/{target_lang_dict[target_lang_key]}{target_class}-topics{max_n_topics}-{term_type}{accent_status}.csv"
topics_df.to_csv(hdp15_topics_out, header = False, index = None)

In [42]:
## HDP (n_topics = 45)
import gensim.models
import pyLDAvis.gensim

max_n_topics = 45
hdp45 = gensim.models.HdpModel(corpus, diction, T = max_n_topics, random_state = 1)
vis_data45 = pyLDAvis.gensim.prepare(hdp45, corpus, diction)
pyLDAvis.display(vis_data45)

In [43]:
## save LDAvis output as a html file
lang_dir_name = target_lang_dict[target_lang_key].split()[0]
save_LDAvis = True
if save_LDAvis:
	vis_output = f"results/LDAvis/{lang_dir_name}/{target_lang_dict[target_lang_key]}{target_class}-HDP-max_ntop{max_n_topics}-{term_type}{accent_status}.html"
	pyLDAvis.save_html(vis_data45, vis_output)

In [44]:
## save topic structures
#hdp.get_topics() # =/= show_topics()
#hdp.print_topics()
hdp_topics = hdp45.show_topics(num_topics = max_n_topics,
                               num_words = n_terms_to_save, formatted = False)
hdp_dict = { tid: values for tid, values in hdp_topics }
## convert to Pandas dataframe
topics_df = pd.DataFrame.from_dict(hdp_dict)
#hdp45_topics_out = f"results/terms-by-topics-raw/hdp{max_n_topics}_topics_raw.csv"
hdp45_topics_out = f"results/terms-by-topics-raw/{lang_dir_name}/{target_lang_dict[target_lang_key]}{target_class}-topics{max_n_topics}-{term_type}{accent_status}.csv"
topics_df.to_csv(hdp45_topics_out, header = False, index = None)

In [45]:
## topic investigation
import numpy as np
import HDP_helper

target_hdp = hdp45
documents_topics = np.zeros([target_hdp.m_T, len(corpus)])
for doc_id, c in enumerate(corpus):
    for topic_id, prob in target_hdp[c]:
        documents_topics[topic_id][doc_id] = prob
#
target_hdp.optimal_ordering()
for topic_id, probs in enumerate(documents_topics):
    print(f"==============")
    topic_t = target_hdp.print_topic(topic_id, topn = n_terms_to_show)
    print(f"topic_id {topic_id}: {HDP_helper.reformat_topic (topic_t, n_terms_to_show)}")
    print(f"nonzero count: {len(probs.nonzero()[0])}")
    for doc_id in probs.argsort()[::-1][:n_docs_to_show]:
        doc = doc_dict[doc_id]
        print(f"\t{probs[doc_id]:0.4f}: {doc}")

topic_id 0: 0.0 * t…al + 0.0 * e…in + 0.0 * a…al + 0.0 * a…i…g + 0.0 * e…i…g + 0.0 * e…ng + 0.0 * r…ng + 0.0 * n…g + 0.0 * a…in + 0.0 * e…ing + 0.0 * a…ng + 0.0 * a…i…l + 0.0 * a…ing + 0.0 * i…e…t + 0.0 * n…al + 0.0 * e…t…l + 0.0 * e…e…t + 0.0 * r…in + 0.0 * tic + 0.0 * e…i…l + 0.0 * t…g + 0.0 * r…i…g + 0.0 * i…nt + 0.0 * ia + 0.0 * h…n + 0.0 * m…c + 0.0 * i…in + 0.0 * f…t + 0.0 * n…in + 0.0 * n…e…t + 0.0 * f…n + 0.0 * n…ng + 0.0 * s…g + 0.0 * r…ti + 0.0 * i…i…g + 0.0 * o…in + 0.0 * r…ing + 0.0 * e…nt + 0.0 * c…al + 0.0 * ma + 0.0 * er…e + 0.0 * t…ng + 0.0 * t…ic + 0.0 * c…g + 0.0 * c…in + 0.0 * l…g + 0.0 * a…t…c + 0.0 * e…en + 0.0 * i…te + 0.0 * et + 0.0 * n…i…g + 0.0 * i…ng + 0.0 * d…s + 0.0 * d…r + 0.0 * i…ing + 0.0 * t…s + 0.0 * o…ng + 0.0 * n…e…e + 0.0 * p…ic + 0.0 * h…r
nonzero count: 2202
	0.9986: inferential
	0.9986: distressing
	0.9985: operational
	0.9985: consonantal
	0.9985: contrasting
	0.9985: interracial
	0.9985: directional
	0.9985: incontinent
	0.9985: secretarial
	0.9

In [46]:
## HDP (n_topics = 90)
import gensim.models
import pyLDAvis.gensim

max_n_topics = 90
hdp90 = gensim.models.HdpModel(corpus, diction, T = max_n_topics,
                               random_state = 1)
vis_data90 = pyLDAvis.gensim.prepare(hdp90, corpus, diction)
pyLDAvis.display(vis_data90)

In [47]:
## save LDAvis output as a html file
save_LDAvis = True
if save_LDAvis:
	lang_dir_name = target_lang_dict[target_lang_key].split()[0]
	vis_output = f"results/LDAvis/{lang_dir_name}/{target_lang_dict[target_lang_key]}{target_class}-HDP-max_ntop{max_n_topics}-{term_type}{accent_status}.html"
	pyLDAvis.save_html(vis_data90, vis_output)

In [48]:
## save topic structures
hdp_topics = hdp90.show_topics(num_topics = max_n_topics,
                               num_words = n_terms_to_save, formatted = False)
hdp_dict = { tid: values for tid, values in hdp_topics }

## convert to Pandas dataframe
topics_df = pd.DataFrame.from_dict(hdp_dict)
hdp90_topics_out = f"results/terms-by-topics-raw/{lang_dir_name}/{target_lang_dict[target_lang_key]}{target_class}-topics{max_n_topics}-{term_type}{accent_status}.csv"
topics_df.to_csv(hdp90_topics_out, header = False, index = None)

In [49]:
## topic investigation
import numpy as np
import HDP_helper

target_hdp = hdp90
documents_topics = np.zeros([target_hdp.m_T, len(corpus)])
for doc_id, c in enumerate(corpus):
    for topic_id, prob in target_hdp[c]:
        documents_topics[topic_id][doc_id] = prob

## investigate topics
target_hdp.optimal_ordering()
for topic_id, probs in enumerate(documents_topics):
    print(f"==============")
    #topic_encoding = ", ".join(hdp.show_topic(topic_id))
    topic_t = target_hdp.print_topic(topic_id, topn = n_terms_to_show)
    print(f"topic_id {topic_id}: {HDP_helper.reformat_topic (topic_t, n_terms_to_show)}")
    print(f"nonzero count: ", len(probs.nonzero()[0]))
    for doc_id in probs.argsort()[::-1][:n_docs_to_show]:
        doc = doc_dict[doc_id]
        print(f"\t{probs[doc_id]:0.4f}: {doc}")

topic_id 0: 0.0 * co…e + 0.0 * un…d + 0.0 * a…al + 0.0 * u…a…e + 0.0 * n…al + 0.0 * t…al + 0.0 * c…n…e + 0.0 * on…e + 0.0 * n…e…e + 0.0 * e…i…l + 0.0 * un…ed + 0.0 * s…o + 0.0 * c…i…e + 0.0 * n…r…e + 0.0 * m…c + 0.0 * a…i…g + 0.0 * h…r + 0.0 * s…ic + 0.0 * n…i…d + 0.0 * n…t…d + 0.0 * n…te + 0.0 * tic + 0.0 * o…i…e + 0.0 * r…ti + 0.0 * u…i…e + 0.0 * a…in + 0.0 * h…l + 0.0 * n…e…t + 0.0 * p…i…e + 0.0 * ia + 0.0 * s…a…e + 0.0 * nd + 0.0 * i…te + 0.0 * n…g + 0.0 * h…t + 0.0 * co…t + 0.0 * ma + 0.0 * d…c + 0.0 * co…n + 0.0 * o…a…e + 0.0 * n…le + 0.0 * p…ic + 0.0 * a…i…l + 0.0 * h…n + 0.0 * n…e…d + 0.0 * n…n…d + 0.0 * c…al + 0.0 * e…te + 0.0 * u…r…e + 0.0 * n…r…d + 0.0 * i…i…e + 0.0 * re…t + 0.0 * on…l + 0.0 * s…g + 0.0 * c…e…t + 0.0 * ch + 0.0 * a…ng + 0.0 * c…nt + 0.0 * s…s + 0.0 * r…in
nonzero count:  1635
	0.9985: uninhibited
	0.9985: conditional
	0.9985: unconcerned
	0.9985: consonantal
	0.9985: conditioned
	0.9985: incontinent
	0.9985: uncluttered
	0.9985: unchartered
	0.9984: unconvin